In [12]:
import tfidf_matcher as tm
import pandas as pd
import numpy as np
from concurrent.futures import ProcessPoolExecutor, as_completed

In [13]:
# Workstation Path
nets_all_data = pd.read_stata('S:/Users/Hiwi/Maris/name_matching/Data/hq_all_maris.dta')
nets_pub_data = pd.read_stata('S:/Users/Hiwi/Maris/name_matching/Data/hq_companies_public_maris.dta')
compustat_data = pd.read_stata('S:/Users/Hiwi/Maris/name_matching/Data/compustat.dta')

# Mac Path
# compustat_data = pd.read_stata('/Users/marisbuttmann/Desktop/HIWI_code/geocode_project/all_data/compustat.dta')
# nets_pub_data = pd.read_stata('/Users/marisbuttmann/Desktop/HIWI_code/geocode_project/all_data/hq_companies_public_maris.dta')
#nets_all_data = pd.read_stata('/Users/marisbuttmann/Desktop/HIWI_code/geocode_project/all_data/hq_all_maris.dta')

In [33]:
compustat = compustat_data['hqcompany']
nets = nets_pub_data['hqcompany']

substitutions = [['\\bCORP\\b', 'CORPORATION'], ['\\bLTD\\b', 'LIMITED'], ['\\bINC\\b', 'INCORPORATED'], ['\\bCO\\b', 'COMPANY'], ['\\bLLC\\b', 'LIMITEDLIABILITYCOMPANY'], ['\\bPLC\\b', 'PUBLICLIMITEDCOMPANY'], ['\\bINTL\\b', 'INTERNATIONAL']]
for sub in substitutions:
    compustat = compustat.str.replace(sub[0], sub[1], regex=True)
    nets = nets.str.replace(sub[0], sub[1], regex=True)

#Check if any inc left in compustat
compustat[compustat.str.contains('INC', na=False)]

0             A & E PLASTIK PAK INCORPORATED
1           A & M FOOD SERVICES INCORPORATED
3        A.A. IMPORTING COMPANY INCORPORATED
5             A.B.A. INDUSTRIES INCORPORATED
6                      ABC INDS INCORPORATED
                        ...                 
42963              CONTEXTLOGIC INCORPORATED
42964               IMMUNITYBIO INCORPORATED
42968     NEXTPLAY TECHNOLOGIES INCORPORATED
42970     INDAPTUS THERAPEUTICS INCORPORATED
42971        CLEARMIND MEDICINE INCORPORATED
Name: hqcompany, Length: 18837, dtype: object

In [14]:
def matching(compustat_names, nets_names):
    workers = 12
    results = pd.DataFrame(columns=['Original Name', 'Lookup 1', 'Lookup 1 Confidence', 'Lookup 1 Index', 'Lookup 2', 'Lookup 2 Confidence', 'Lookup 2 Index',])

    # Split nets_all_data into 4 pieces
    nets_splits = np.array_split(nets_names, workers)

    with ProcessPoolExecutor(max_workers=workers) as executor:
        # Submit a task for each split of nets_all_data
        futures = [executor.submit(tm.matcher, compustat_names, data_split, k_matches=2, ngram_length=3) for data_split in nets_splits]

        for future in as_completed(futures):
            try:
               print(future.result())
               results = pd.concat([future.result(), results])
            except Exception as e:
                print(e)

    return results

In [34]:
# result = matching(compustat, nets)
result = tm.matcher(compustat, nets, k_matches=2, ngram_length=3)
#pd.DataFrame(result)
result

,Original Name,Lookup 1,Lookup 1 Confidence,Lookup 1 Index,Lookup 2,Lookup 2 Confidence,Lookup 2 Index
0,A & E PLASTIK PAK INCORPORATED,CARLISLE PLASTICS INCORPORATED ...,0.22,12056,AIR COMPAK INTERNATIONAL INCORPORATED ...,0.22,15056
1,A & M FOOD SERVICES INCORPORATED,F & M FINANCIAL SERVICES CORPORATION ...,0.46,11382,SUPER FOOD SERVICES INCORPORATED ...,0.43,12218
2,AAI CORPORATION,DESAI CAPITAL MANAGEMENT INCORPORATED ...,0.23,21041,MCI COMMUNICATIONS CORPORATION ...,0.20,5463
3,A.A. IMPORTING COMPANY INCORPORATED,A A IMPORTING COMPANY INCORPORATED ...,0.58,16774,PROMUS OPERATING COMPANY INCORPORATED ...,0.26,19683
4,AAR CORPORATION,AAR CORPORATION ...,0.30,1034,CYBERSTAR COMPUTER CORPORATION ...,0.28,4581
...,...,...,...,...,...,...,...
42974,IVECO GROUP N V,TCO GROUP HOLDINGS LP ...,0.21,16865,ALI GROUP NORTH AMERICA CORPORATION ...,0.21,5183
42975,DAIMLER TRUCK HOLDING AG,FREYMILLER TRUCKING INCORPORATED ...,0.26,13684,ROLLINS TRUCK LEASING CORPORATION ...,0.22,4206
42976,COOL COMPANY LIMITED,HOTEL ACQUISITION COMPANY LIMITEDLIABILITYCOMP...,0.37,9625,HT INTERMEDIATE COMPANY LIMITEDLIABILITYCOMPAN...,0.37,21646
42977,HALEON PUBLICLIMITEDCOMPANY,GATES INDUSTRIAL CORPORATION PUBLICLIMITEDCOMP...,0.55,5876,ENDO INTERNATIONAL PUBLICLIMITEDCOMPANY ...,0.49,9342


In [38]:
result[result['Lookup 1 Confidence'] > 0.5].sort_values('Lookup 1 Confidence', ascending=True).shape

(14032, 7)